Обучить СНС с помощью Transfer Learning на датасете Food-101
Использовать тонкую настройку существующей предобученной модели и методы аугментации данных.

In [0]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import backend as K

In [0]:
(train_ds, test_ds, valid_ds), ds_info = tfds.load(
    'food101',
    as_supervised=True,
    with_info=True,
    split=['train[:90%]', 'train[90%:]', 'validation']
    )


In [0]:
def augment(image, label):
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.resize(image, (INP_SIZE, INP_SIZE))
  return image, label

def prepare(image, label):
  image = tf.image.convert_image_dtype(image, tf.float32)
  return tf.image.resize(image, (INP_SIZE, INP_SIZE)), label


In [0]:
INP_SIZE = 64
NUM_EPOCHS = 10
BATCH_SIZE = 224



train_ds = train_ds.shuffle(buffer_size=3000)
train_ds = train_ds.map(augment)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)

test_ds = test_ds.shuffle(buffer_size=3000)
test_ds = test_ds.map(prepare)
test_ds = test_ds.batch(128, drop_remainder=True)

In [0]:
out = ds_info.features['label'].num_classes

In [36]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(INP_SIZE, INP_SIZE, 3),
    include_top=False,
    weights='imagenet',
)

base_model.trainable = True 

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(out, activation='softmax'),
])

In [0]:
LEARNING_RATE = 0.0001
optimizer = tf.keras.optimizers.RMSprop(lr=LEARNING_RATE)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=test_ds)

Epoch 1/10
304/304 [==============================] - 147s 484ms/step - loss: 4.1956 - accuracy: 0.0979 - val_loss: 5.6893 - val_accuracy: 0.1082
Epoch 2/10
304/304 [==============================] - 148s 488ms/step - loss: 3.1540 - accuracy: 0.2569 - val_loss: 6.1199 - val_accuracy: 0.1392
Epoch 3/10
304/304 [==============================] - 148s 486ms/step - loss: 2.7087 - accuracy: 0.3404 - val_loss: 5.0068 - val_accuracy: 0.1891
Epoch 4/10
304/304 [==============================] - 150s 492ms/step - loss: 2.4271 - accuracy: 0.3965 - val_loss: 4.1617 - val_accuracy: 0.2229
Epoch 5/10
304/304 [==============================] - 148s 486ms/step - loss: 2.2106 - accuracy: 0.4392 - val_loss: 3.6178 - val_accuracy: 0.2627
Epoch 6/10
304/304 [==============================] - 149s 490ms/step - loss: 2.0256 - accuracy: 0.4823 - val_loss: 3.1690 - val_accuracy: 0.3026
Epoch 7/10
304/304 [==============================] - 148s 486ms/step - loss: 1.8494 - accuracy: 0.5247 - val_loss: 3.0297 -